In [145]:
import sys

sys.path.append('/home/anton/Documents/Tulane/Research/demo-vae/pip/src/')

from demovae import DemoVAE

field = 'Total_Drinks_7days'

vae = DemoVAE(pperiod=50, use_cuda=True, nepochs=5000, latent_dim=100, weight_decay=0, lr=5e-5, loss_rec_mult=1e3,
              loss_pred_mult=0.01, loss_C_mult=1e4)
vae.load(f'/home/anton/Documents/Yale/HCP/VAE_Models/456_subs/{field}.pkl')

print('Done')

Done


In [146]:
import pickle

dct = pickle.load(open(f'/home/anton/Documents/Yale/HCP/VAE_Models/456_subs/demos_{field}.pkl', 'rb'))

print(list(dct.keys()))
print('Done')

['demos', 'types']
Done


In [147]:
# Load ADNI model

import pickle

adni = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/ADNI/weights0.pkl', 'rb'))

list(adni.keys())

['intercept', 'coef']

In [148]:
dct['demos'][4][:20]

array([ 9., 10.,  0.,  7.,  3.,  0.,  5.,  0.,  3.,  3.,  5.,  8., 18.,
        0.,  2.,  1., 13.,  8.,  3.,  1.])

In [149]:
import numpy as np

vals = []

for i in range(20):
    synth = vae.transform(len(dct['demos'][0]), dct['demos'], dct['types'])
    
    b = adni['intercept']
    w = adni['coef']
    sum_logits = 0
    
    for fc in synth:
        logits = b+np.sum(fc*w)
        sum_logits += logits
    
    vals.append(sum_logits/len(synth))

print(np.mean(vals))
print(np.std(vals))

-44.173985434931026
0.8165624326557286


In [155]:
# Height and AD https://pmc.ncbi.nlm.nih.gov/articles/PMC6037293/
# Observational studies have linked increased adult height with better cognitive performance and reduced 
# risk of Alzheimer’s disease (AD). It is unclear whether the associations are due to shared biological 
# processes that influence height and AD or due to confounding by early life exposures or environmental factors.

vals = []

for i in range(20):
    ndemo = []
    for d in dct['demos']:
        dd = d+0
        ndemo.append(dd)

    # ndemo[1] = 0*np.ones(len(ndemo[1]))
    # ndemo[3] += 15
    # ndemo[4] = 0*np.ones(len(ndemo[4]))
    ndemo[4] += 15
    
    nsynth = vae.transform(len(dct['demos'][0]), ndemo, dct['types'])
    
    sum_logits = 0
    
    for fc in nsynth:
        logits = b+np.sum(fc*w)
        sum_logits += logits
    
    vals.append(sum_logits/len(nsynth))

print(np.mean(vals))
print(np.std(vals))

-45.257404553809174
0.7996685867798532
